# Signaling Adversity

(_A chapter name after my_ ❤️_.)_

## Nothing

A very common datatype in Haskell because it lets us return a default `Nothing` value when we don’t have any sensible values to return for our intended type `a`:

```haskell
data Maybe a = Nothing | Just a
```

Below is a function that is explicit about the possibility of not getting a result when the `Integer` provided is odd.

In [ ]:
ifEvenAdd2 :: Integer -> Maybe Integer
ifEvenAdd2 n =
    if even n
    then Just (n + 2)
    else Nothing

--

ifEvenAdd2 12
ifEvenAdd2 13

## Smart Constructors

By making use of `Maybe`.

In [ ]:
type Name = String
type Age = Integer

data Person =
    Person Name Age
    deriving Show
    
mkPerson :: Name -> Age -> Maybe Person
mkPerson name age
    | name /= "" && age >= 0 =
        Just $ Person name age
    | otherwise = Nothing

--

mkPerson "John Browning" 160
mkPerson "" 160
mkPerson "blah" 0
mkPerson "blah" (-9001)

## Either

A way to know why our inputs were incorrect, if they were incorrect.

```haskell
data Either a b = Left a | Right b
```

It is conventional to use `Left` as our invalid or error constructor.

In [ ]:
-- Sum type for an invalid person
data PersonInvalid =
    NameEmpty
    | AgeTooLow
    deriving (Eq, Show)

mkPerson :: Name -> Age -> Either PersonInvalid Person
mkPerson name age
    | name /= "" && age >= 0 = Right $ Person name age
    | name == ""             = Left NameEmpty
    | otherwise              = Left AgeTooLow

--

mkPerson "Djali" 5
mkPerson "" 10
mkPerson "Djali" (-1)
mkPerson "" (-1)

## List of Validaton Errors

Man, all of this to have `mkPerson` return a list of validation errors. Also, thanks for the `liftA2` tease.

In [ ]:
-- Type alias for Either [PersonInvalid] a
type ValidatePerson a = Either [PersonInvalid] a

ageOkay :: Age -> Either [PersonInvalid] Age
ageOkay age =
    case age >= 0 of
        True -> Right age
        False -> Left [AgeTooLow]

nameOkay :: Name -> Either [PersonInvalid] Name
nameOkay name =
    case name /= "" of
        True -> Right name
        False -> Left [NameEmpty]

mkPerson :: Name -> Age -> ValidatePerson Person
mkPerson name age =
    mkPerson' (nameOkay name) (ageOkay age)

mkPerson' :: ValidatePerson Name -> ValidatePerson Age -> ValidatePerson Person
mkPerson' (Right nameOk) (Right ageOk) =
    Right (Person nameOk ageOk)
mkPerson' (Left badName) (Left badAge) =
    Left (badName ++ badAge)
mkPerson' (Left badName) _ = Left badName
mkPerson' _ (Left badAge) = Left badAge

--

mkPerson "" (-1)
mkPerson "Joker" (-1)
mkPerson "Joker" 25

## Kinds and Higher-kinded Types

_Type constructors_ (that is, higher-kinded types) are types that take more types as arguments. _Type constants_ refer to types that take no arguments and are already types.

The following is an example of a type that has a type constructor rather than a type constant:

```haskell
data Example a = Blah | RoofGoats | Woot a
```

`Example` is a _type constructor_ rather than a _type constant_ because it takes a type argument `a` which is used with the `Woot` data constructor.

---

The `Maybe` and `Either` datatypes we’ve just reviewed also have _type constructors_ rather than _type constants_.

```haskell
Prelude> :k Maybe
Maybe :: * -> *
Prelude> :k Maybe Int
Maybe Int :: *

Prelude> :k Either
Either :: * -> * -> *
Prelude> :k Either Int
Either Int :: * -> *
Prelude> :k Either Int String
Either Int String :: *
```

## Lifted and Unlifted Types

`kind *` is the kind of all standard lifted types, while types that have the `kind #` are unlifted. A lifted type is any that can be inhabited by _bottom_. Unlifted types are any type which cannot be inhabited by _bottom_.

Goal is to give us a way to understand when type errors are caused by things not being fully applied? `¯\_(ツ)_/¯`

```haskell
Prelude> :k Maybe
Maybe :: * -> *
Prelude> :k Maybe Int
Maybe Int :: *
Prelude> :k Maybe Bool
Maybe Bool :: *

Prelude> :k Maybe Maybe

Expecting one more argument to ‘Maybe’
The first argument of ‘Maybe’ should have kind ‘*’,
    but ‘Maybe’ has kind ‘* -> *’
In a type in a GHCi command: Maybe Maybe

Prelude> :k Maybe Char
Maybe Char :: *
Prelude> :k Maybe (Maybe Char)
Maybe (Maybe Char) :: *
```

## Data Constructors are Functions

Some examples of how data constructros can act like functions:

```haskell
Prelude> data Trivial = Trivial deriving Show
Prelude> Trivial 1

Couldn't match expected type ‘Integer -> t’
    with actual type ‘Trivial’
    
Prelude> data UnaryC = UnaryC Int deriving Show
Prelude> :t UnaryC UnaryC :: Int -> UnaryC
Prelude> UnaryC 10 UnaryC 10
Prelude> :t UnaryC 10 UnaryC 10 :: UnaryC

Prelude> data Unary a = Unary a deriving Show
Prelude> :t Unary Unary :: a -> Unary a
Prelude> :t Unary 10 Unary 10 :: Num a => Unary a
Prelude> :t Unary "blah" Unary "blah" :: Unary [Char]
```

# Exercises

1) Given

```haskell
id :: a -> a
```

What is the kind of `a`? `*`

2) Given

```haskell
r :: a -> f a
```

What are the kinds of `a` and `f`? `* -> *`

## String Processing

1) Write a recursive function named `replaceThe` which takes a text/string, breaks it into words and replaces each instance of “the” with “a”. It’s intended only to replace exactly the word “the”. `notThe` is a suggested helper function for accomplishing this.

In [ ]:
notThe :: String -> Maybe String
notThe "the" = Nothing
notThe word  = Just word

--

notThe "the"
notThe "blahtheblah"
notThe "woot"

In [ ]:
import Data.Char
import Data.Maybe

replaceThe :: String -> String
replaceThe =
    unwords . map (fromMaybe "a" . notThe) . words

--

replaceThe "the cow loves us"

2) Write a recursive function that takes a text/string, breaks it into words, and counts the number of instances of ”the” followed by a vowel-initial word.

In [ ]:
vowels :: [Char]
vowels = "aeiou"

beforeVowelInitial :: String -> Maybe String
beforeVowelInitial body@('t':'h':'e':' ':x:xs)
    | elem x vowels = Nothing
    | otherwise     = Just body
beforeVowelInitial body = Just body

countTheBeforeVowel :: String -> Integer
countTheBeforeVowel "" = 0
countTheBeforeVowel body =
    case beforeVowelInitial body of
        Nothing     -> 1 + countTheBeforeVowel (drop 4 body)
        Just (x:xs) -> 0 + countTheBeforeVowel xs

--

countTheBeforeVowel "the cow"
countTheBeforeVowel "the evil cow"
countTheBeforeVowel "the evil cow jumped over the ill moon"

3) Return the number of letters that are vowels in a word.

__Hint__: it’s helpful to break this into steps. Add any helper functions necessary to achieve your objectives.

- Test for vowelhood  
- Return the vowels of a string  
- Count the number of elements returned

In [ ]:
countVowels :: String -> Integer
countVowels = toInteger . length . filter (`elem` vowels)

--

countVowels "the cow"
countVowels "Mikolajczak"

## Validate the Word

Use the `Maybe` type to write a function that counts the number of vowels in a string and the number of consonants. If the number of vowels exceeds the number of consonants, the function returns `Nothing`. In many human languages, vowels rarely exceed the number of consonants so when they do, it may indicate the input isn’t a word (that is, a valid input to your dataset):

In [ ]:
newtype Word' =
    Word' String deriving (Eq, Show)

countConsonants :: String -> Integer
countConsonants = toInteger . length . filter (not . (`elem` vowels))

mkWord :: String -> Maybe Word'
mkWord xs
    | vs > cs   = Nothing
    | otherwise = Just (Word' xs)
    where vs = countVowels xs
          cs = countConsonants xs
          
--

mkWord "hello there"
mkWord "helloooooooo there"

## It’s Only Natural

You’ll be presented with a datatype to represent the natural numbers. The only values representable with the naturals are whole numbers from zero to infinity. Your task will be to implement functions to convert `Naturals` to `Integers` and `Integers` to `Naturals`. The conversion from `Naturals` to `Integers` won’t return `Maybe` because `Integer` is a strict superset of `Natural`. Any `Natural` can be represented by an `Integer`, but the same is not true of any `Integer`. Negative numbers are not valid natural numbers.

In [ ]:
data Nat =
    Zero
    | Succ Nat
    deriving (Eq, Show)

natToInteger :: Nat -> Integer
natToInteger Zero = 0
natToInteger (Succ x) = 1 + natToInteger x

--

natToInteger Zero
natToInteger (Succ Zero)
natToInteger (Succ (Succ Zero))

In [ ]:
integerToNat :: Integer -> Maybe Nat
integerToNat 0 = Just Zero
integerToNat n
    | n < 0     = Nothing
    | otherwise = Just (Succ x)
    where (Just x) = integerToNat $ (n - 1)

--

integerToNat 0
integerToNat 1
integerToNat 2
integerToNat (-1)

## Small Library for Maybe

Write the following functions. This may take some time. 😢

1) Simple boolean checks for `Maybe` values.

In [ ]:
isJust' :: Maybe a -> Bool
isJust' Nothing  = False
isJust' _        = True

isNothing' :: Maybe a -> Bool
isNothing' Nothing = True
isNothing' _       = False

--

isJust' (Just 1)
isJust' Nothing

isNothing' (Just 1)
isNothing' Nothing

2) The following is the `Maybe` catamorphism. You can turn a `Maybe` value into anything else with this.

In [ ]:
import Data.Maybe

mayybee :: b -> (a -> b) -> Maybe a -> b
mayybee b f x
    | isJust x  = f (fromJust x)
    | otherwise = b

--

mayybee 0 (+1) Nothing
mayybee 0 (+1) (Just 1)

3) In case you just want to provide a fallback value.

In [ ]:
fromMaybe' :: a -> Maybe a -> a
fromMaybe' a x =
    case x of
        Nothing -> a
        Just v  -> v

--

fromMaybe' 0 Nothing
fromMaybe' 0 (Just 1)

4) Converting between `List` and `Maybe`.

In [ ]:
listToMaybe' :: [a] -> Maybe a
listToMaybe' []     = Nothing
listToMaybe' (x:xs) = Just x

maybeToList' :: Maybe a -> [a]
maybeToList' Nothing  = []
maybeToList' (Just v) = [v]

--

listToMaybe' [1, 2, 3]
listToMaybe' []

maybeToList' (Just 1)
maybeToList' Nothing

5) For when we want to drop the `Nothing` values from our list.

In [ ]:
catMaybes' :: [Maybe a] -> [a]
catMaybes' [] = []
catMaybes' (x:xs) =
    case x of
        Nothing -> catMaybes' xs
        Just v  -> v : catMaybes' xs

--

catMaybes' [Just 1, Nothing, Just 2]
catMaybes' (replicate 3 Nothing)

6) You’ll see this called “sequence” later.

In [ ]:
flipMaybe :: [Maybe a] -> Maybe [a]
flipMaybe xs
    | any isNothing' xs = Nothing
    | otherwise         = Just $ catMaybes' xs

--

flipMaybe [Just 1, Just 2, Just 3]
flipMaybe [Just 1, Nothing, Just 3]

## Small Library for Either

Write each of the following functions. If more than one possible unique function exists for the type, use common sense to determine what it should do.

1) Try to eventually arrive at a solution that uses `foldr`, even if earlier versions don’t use `foldr`.

In [ ]:
import Data.Either

lefts' :: [Either a b] -> [a]
lefts' xs = foldr f [] (filter isLeft xs)
    where f (Left v) acc = v : acc

lefts' [Left 2, Right 5]

2) Same as the last one. Use `foldr` eventually.

In [ ]:
rights' :: [Either a b] -> [b]
rights' xs = foldr f [] (filter isRight xs)
    where f (Right v) acc = v : acc

rights' [Left 2, Right 5]

In [ ]:
partitionEithers' :: [Either a b] -> ([a], [b])
partitionEithers' xs =
    (lefts' xs, rights' xs)

--

partitionEithers' [Left 2, Right 5, Left 1, Left 10]

In [ ]:
eitherMaybe' :: (b -> c) -> Either a b -> Maybe c
eitherMaybe' f (Right b) = Just (f b)
eitherMaybe' _ _         = Nothing

--

eitherMaybe' (+1) (Left 2)
eitherMaybe' (+1) (Right 5)

In [ ]:
either' :: (a -> c) -> (b -> c) -> Either a b -> c
either' f _ (Left v)  = f v
either' _ g (Right v) = g v

--

either' length (*2) (Left "Joker")
either' length (*2) (Right 3)

In [ ]:
eitherMaybe'' :: (b -> c) -> Either a b -> Maybe c
eitherMaybe'' f x = either' (const Nothing) (Just . f) x

--

eitherMaybe'' (+1) (Left 2)
eitherMaybe'' (+1) (Right 5)

## Unfolds

1) Write the function `myIterate` using direct recursion. Compare the behavior with the built-in `iterate` to gauge correctness. Do not look at the source or any examples of `iterate` so that you are forced to do this yourself.

In [ ]:
myIterate :: (a -> a) -> a -> [a]
myIterate f x = x : myIterate f (f x)

--

take 10 (iterate (2*) 1)
take 10 (myIterate (2*) 1)

2) Write the function `myUnfoldr` using direct recursion. Compare with the built-in `unfoldr` to check your implementa-tion. Again, don’t look at implementations of `unfoldr` so that you figure it out yourself.

In [ ]:
import Data.List

myUnfoldr :: (b -> Maybe (a, b)) -> b -> [a]
myUnfoldr f x =
    case f x of
        Nothing     -> []
        Just (a, b) -> a : myUnfoldr f b

--

joker :: Integer -> Maybe (Integer, Integer)
joker x
    | x == 0    = Nothing
    | otherwise = Just (x, x -1)

unfoldr joker 10
myUnfoldr joker 10

3) Rewrite `myIterate` into `betterIterate` using `myUnfoldr`.

__Hint__: We used `unfoldr` to produce the same results as `iterate` earlier. Do this with diﬀerent functions and see if you can abstract the structure out.

```haskell
myUnfoldr :: (b -> Maybe (a, b)) -> b -> [a]
```

In [ ]:
betterIterate :: (a -> a) -> a -> [a]
betterIterate f x = myUnfoldr g x
    where g y = Just (y, f y)

--

take 10 (iterate (2*) 1)
take 10 (betterIterate (2*) 1)

## BinaryTree

1) Write `unfold` for `BinaryTree`.  
2) Make a tree builder using the `unfold` function above.

In [ ]:
data BinaryTree a =
    Leaf
    | Node (BinaryTree a) a (BinaryTree a)
    deriving (Eq, Ord, Show)

unfold :: (a -> Maybe (a,b,a)) -> a -> BinaryTree b
unfold f x =
    case f x of
        Nothing        -> Leaf
        Just (x, y, z) -> Node (unfold f x) y (unfold f z)

treeBuild :: Integer -> BinaryTree Integer
treeBuild n = unfold f 0
    where f x
            | x == n    = Nothing
            | otherwise = Just (x + 1, x, x + 1)

--

treeBuild 0
treeBuild 1
treeBuild 2
treeBuild 3